# softmax回归的从零开始实现

In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 初始化模型参数

和之前线性回归的例子一样，这里的每个样本都将用固定长度的向量表示。 原始数据集中的每个样本都是28*28的图像。 本节将展平每个图像，把它们看作长度为784的向量。在softmax回归中，我们的输出与类别一样多。 因为我们的数据集有10个类别，所以网络输出维度为10。 因此，权重将构成一个的矩阵， 偏置将构成一个的行向量。 与线性回归一样，我们将使用正态分布初始化我们的权重W，偏置初始化为0。

In [2]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

In [4]:
X = torch.tensor([[1, 2, 3], [4, 5, 6]])

X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5, 7, 9]]),
 tensor([[ 6],
         [15]]))

In [5]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    return X_exp / partition # 应用广播机制

In [9]:
"""广播机制案例演示"""
a = torch.tensor([[4, 4], [6, 6]])

x_exp = torch.exp(a)
pa = x_exp.sum(1, keepdim=True)

x_exp, pa, x_exp / pa

(tensor([[ 54.5981,  54.5981],
         [403.4288, 403.4288]]),
 tensor([[109.1963],
         [806.8576]]),
 tensor([[0.5000, 0.5000],
         [0.5000, 0.5000]]))

这个案例中的pa在被除以x_exp时，pa的维度为(2,1)，x_exp的维度为(2,2)，由于pa的维度为(2,1)，所以pa被广播为(2,2)，然后进行除法运算。
pa会复制一列，使得pa的维度和x_exp的维度一致，然后进行除法运算。

In [ ]:
"""定义模型"""
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)

In [10]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]

tensor([0.1000, 0.5000])

在 PyTorch 中，y_hat 是一个二维张量（tensor），其形状为 (2, 3)，即有 2 行 3 列。y 是一个一维张量，包含两个元素 [0, 2]。

当你执行 y_hat[[0, 1], y] 这个操作时，你正在使用高级索引（fancy indexing）来选择 y_hat 张量中的特定元素。

这里的索引操作可以分解为两部分：

y_hat[[0, 1]]：这部分选择了 y_hat 的第 0 行和第 1 行，结果是一个形状为 (2, 3) 的张量。

y：这个一维张量包含了两个索引，分别是 0 和 2。

将这两部分结合起来，y_hat[[0, 1], y] 会从 y_hat 的第 0 行和第 1 行中，根据 y 中的索引选择元素。具体来说：

从第 0 行中选择索引为 0 的元素，即 y_hat[0, 0]。
从第 1 行中选择索引为 2 的元素，即 y_hat[1, 2]。
因此，执行 y_hat[[0, 1], y] 后，你将得到一个包含这两个元素的新张量。在这个例子中，y_hat[[0, 1], y] 的结果将是 [0.1, 0.5]。这是因为：

y_hat[0, 0] 是 0.1。
y_hat[1, 2] 是 0.5。
这个操作在机器学习中常用于从预测张量中提取实际标签对应的预测值。

In [11]:
"""交叉熵损失函数"""
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])